#### convert data for bokeh plot

In [1]:
import joblib
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
results_compare = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/results_compare_adjusted_scaled.joblib.compressed')

In [3]:
def geodataframe_to_columndatasource_df(
    gdf, 
    polygon_name_column='NAME_1',
    scenario_variable='RES1.0_IND1.0_TRA1.0_AGR1.0_ENE1.0'):
    xs = []
    ys = []
    for obj in gdf.geometry.boundary:
        if obj.type == 'LineString':
            obj_x, obj_y = obj.xy
            xs.append([[list(obj_x)]])
            ys.append([[list(obj_y)]])
        elif obj.type == 'MultiLineString':
            obj_x = []
            obj_y = []
            for line in obj:
                line_x, line_y = line.xy
                obj_x.append([list(line_x)])
                obj_y.append([list(line_y)])
            xs.append(obj_x)
            ys.append(obj_y)

    locations = gdf[polygon_name_column].values
    df = pd.DataFrame({
        'location': locations,
        'xs': xs, 
        'ys': ys, 
        'scenario_variable': gdf[scenario_variable]})
    for col in [col for col in gdf.columns if 'RES' in col]:
        df[col] = gdf[col]
    return df

In [ ]:
# source per outcome
source_dfs = {}
options = []

sims = list(results_compare['China'][output][outcome][metric].keys())

for outcome in ['exposure', 'mort', 'dalys_rate']:
    for metric in ['mean', 'lower', 'upper']:
        if outcome == 'exposure' and metric != 'mean':
            continue
        else:
            for output in ['PM2_5_DRY', 'o3_6mDM8h']:
                gdf = gpd.read_file('/nfs/a68/earlacoa/shapefiles/china/gadm36_CHN_1.shp')

                for region in gdf.NAME_1.unique():
                    for sim in sims:
                        options.append(f'{sim}')
                        result = results_compare[region][output][outcome][metric][sim]
                        gdf.loc[gdf.NAME_1 == region, f'{sim}'] = result

                source_df = geodataframe_to_columndatasource_df(
                    gdf, 
                    polygon_name_column='NAME_1', 
                    scenario_variable='RES1.0_IND1.0_TRA1.0_AGR1.0_ENE1.0')
                source_dfs.update({f'{output}_{outcome}_{metric}': source_df})
    
options = sorted(list(set(options)))

In [ ]:
joblib.dump(source_dfs, '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_adjusted_scaled.joblib.compressed', compress=True)
joblib.dump(options, '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/options.joblib.compressed', compress=True)

In [ ]:
source_dfs = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_adjusted_scaled.joblib.compressed')
options = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/options.joblib.compressed')

In [ ]:
source_dfs_exposure = {key: source_dfs[key] for key in ['PM2_5_DRY_exposure_mean', 'o3_6mDM8h_exposure_mean']}
source_dfs_mort = {key: source_dfs[key] for key in ['PM2_5_DRY_mort_mean', 'o3_6mDM8h_mort_mean']}

In [ ]:
joblib.dump(source_dfs_exposure, '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_exposure_adjusted_scaled.joblib.compressed', compress=True)
joblib.dump(source_dfs_mort, '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_mort_adjusted_scaled.joblib.compressed', compress=True)

In [4]:
source_dfs_exposure = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_exposure_adjusted_scaled.joblib.compressed')
source_dfs_mort = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_mort_adjusted_scaled.joblib.compressed')

In [41]:
source_dfs_exposure_float16 = {}
for key, df in source_dfs_exposure.items():
    source_dfs_exposure_float16.update({key: df.astype({column: np.float16 for column in list(df.columns[4:])})})
    

source_dfs_mort_float16 = {}
for key, df in source_dfs_mort.items():
    source_dfs_mort_float16.update({key: df.astype({column: np.float16 for column in list(df.columns[4:])})}) 

In [46]:
source_dfs_mort_float16['PM2_5_DRY_mort_mean'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Columns: 32772 entries, location to RES1.4_IND1.4_TRA1.4_AGR1.4_ENE1.4
dtypes: float16(32768), float64(1), object(3)
memory usage: 1.9+ MB


In [45]:
source_dfs_mort['PM2_5_DRY_mort_mean'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Columns: 32772 entries, location to RES1.4_IND1.4_TRA1.4_AGR1.4_ENE1.4
dtypes: float64(32769), object(3)
memory usage: 7.8+ MB


In [47]:
joblib.dump(source_dfs_exposure_float16, '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_exposure_adjusted_scaled_float16.joblib.compressed', compress=True)
joblib.dump(source_dfs_mort_float16, '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_mort_adjusted_scaled_float16.joblib.compressed', compress=True)

['/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_mort_adjusted_scaled_float16.joblib.compressed']

In [ ]:
source_dfs_exposure_float16 = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_exposure_adjusted_scaled_float16.joblib.compressed')
source_dfs_mort_float16 = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_mort_adjusted_scaled_float16.joblib.compressed')

In [50]:
source_dfs_exposure_float16_pm25only = {k: v for k, v in source_dfs_exposure_float16.items() if 'PM2_5_DRY' in k}

In [51]:
source_dfs_mort_float16_pm25only = {k: v for k, v in source_dfs_mort_float16.items() if 'PM2_5_DRY' in k}

In [52]:
joblib.dump(source_dfs_exposure_float16_pm25only, '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_exposure_adjusted_scaled_float16_pm25only.joblib.compressed', compress=True)
joblib.dump(source_dfs_mort_float16_pm25only, '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_mort_adjusted_scaled_float16_pm25only.joblib.compressed', compress=True)

['/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_mort_adjusted_scaled_float16_pm25only.joblib.compressed']

In [ ]:
source_dfs_exposure_float16_pm25only = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_exposure_adjusted_scaled_float16_pm25only.joblib.compressed')
source_dfs_mort_float16_pm25only = joblib.load('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/health_impact_assessments/source_dfs_mort_adjusted_scaled_float16_pm25only.joblib.compressed')